In [39]:
import geopandas as gpd
import requests
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon
import fontawesome as fa 
from geopy.distance import geodesic

In [40]:
search_query = "Pharmacie"
city_name = "Youssoufia"
region = "Marrakech-Safi"

url = f"https://nominatim.openstreetmap.org/search?q={search_query},{city_name},{region}&format=json"

# Fetch data from OpenStreetMap API (Nominatim)
response = requests.get(url)
data = response.json()

# Extract coordinates and hospital names
hospitals = []
for place in data:
    hospital_name = place.get("display_name", "Unknown Hospital")
    lon, lat = place["lon"], place["lat"]
    hospitals.append({"name": hospital_name, "lon": lon, "lat": lat})

# Create a GeoDataFrame with the geometry column
gdf = gpd.GeoDataFrame(hospitals, geometry=gpd.points_from_xy([float(hospital["lon"]) for hospital in hospitals], 
                                                             [float(hospital["lat"]) for hospital in hospitals]),
                       crs="EPSG:4326")

display(gdf)


,name,lon,lat,geometry
0,"Pharmacie Moderne الصيدلية الحديثة, RN11, Yous...",-8.5371963,32.2415859,POINT (-8.53720 32.24159)
1,"Pharmacie Dokkali صيدلية دكالي, RN11, Youssouf...",-8.5291738,32.2566127,POINT (-8.52917 32.25661)
2,"Pharmacie Ibn Sina صيدلية ابن سينا, RN11, Yous...",-8.5422871,32.239898,POINT (-8.54229 32.23990)
3,"Pharmacie Kachkat صيدلية كاشكاط, RN11, Youssou...",-8.5332465,32.2458067,POINT (-8.53325 32.24581)
4,"Pharmacie Al Qods صيدلية القدس, RN11, Youssouf...",-8.5358955,32.2538836,POINT (-8.53590 32.25388)
5,"Pharmacie Annour صيدلية النور, RN11, Youssoufi...",-8.5326451,32.237092,POINT (-8.53265 32.23709)
6,"Pharmacie Lahdir صيدلية لهضير, RN11, Youssoufi...",-8.532388,32.2489358,POINT (-8.53239 32.24894)
7,"Pharmacie Louis Gentil صيدلية لويس جونتي, RN11...",-8.5283145,32.246148,POINT (-8.52831 32.24615)
8,"Pharmacie Najmi صيدلية نجمي, RN11, Youssoufia,...",-8.5342218,32.2515064,POINT (-8.53422 32.25151)
9,"Pharmacie Al Amal صيدلية الأمل, RN11, Youssouf...",-8.5328651,32.2353451,POINT (-8.53287 32.23535)


In [41]:
gdf.to_csv('data.csv', index = False)

In [48]:
# Read data from CSV file and create GeoDataFrame
data = pd.read_csv("merged_data.csv")
gdf = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy(data['lon'], data['lat']), crs="EPSG:4326")

def calculate_distance(coord1, coord2):
    point1 = (coord1[1], coord1[0])  
    point2 = (coord2[1], coord2[0]) 
    return geodesic(point1, point2).kilometers

# Create a Folium map centered at the specified location, using satellite tiles
map_center = [32.25, -8.52]
my_map = folium.Map(location=map_center, zoom_start=12, tiles='OpenStreetMap', control_scale=True)

# Create a MarkerCluster layer to group the markers
marker_cluster = MarkerCluster().add_to(my_map)

# Add markers to the map (same as before)
for idx, row in gdf.iterrows():
    name = row["name"]
    lat = row["lat"]
    lon = row["lon"]
    popup_text = f"<b>{name}</b><br>Latitude: {lat}<br>Longitude: {lon}"

    # Check the name for different categories (case-insensitive)
    if "pharmacie" in row["name"].lower():
        icon_color = "green"
        icon_html = '<i class="fa fa-hospital" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "gare" in row["name"].lower():
        icon_color = "red"
        icon_html = '<i class="fa fa-train" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "youcode" in row["name"].lower():
        icon_color = "blue"
        icon_html = '<i class="fa fa-graduation-cap" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "masjid" in row["name"].lower():
        icon_color = "orange"
        icon_html = '<i class="fa fa-mosque" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "souk" in row["name"].lower():
        icon_color = "purple"
        icon_html = '<i class="fa fa-shopping-basket" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "hôtel" in row["name"].lower():
        icon_color = "brown"
        icon_html = '<i class="fa fa-hotel" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    elif "jardin" in row["name"].lower():
        icon_color = "#66ff66"
        icon_html = '<i class="fa fa-tree" style="color: {}; font-size: 30px;"></i>'.format(icon_color)
    else:
        icon_color = "gray"
        icon_html = '<i class="fa fa-info-circle" style="color: {}; font-size: 30px;"></i>'.format(icon_color)

    folium.Marker(location=[lat, lon], popup=folium.Popup(popup_text),
                  icon=DivIcon(icon_size=(30, 30), icon_anchor=(15, 15), html=icon_html)).add_to(marker_cluster)

# Extract place names from the legend
place_names = ["Pharmacie Chefchaouni", "Gare de Youssoufia", "Pharmacie Inas", "Pharmacie Ahmar", "Pharmacie Zaid",
               "Youcode", "souk baladi", "Hôtel de Youssoufia", "Jardin boulevard hassan II"]
# Filter the data to include only the specified place names
line_data = gdf[gdf["name"].str.contains('|'.join(place_names), case=False)]

# Create a LineString to draw on the map
line_coords = [(lat, lon) for lat, lon in zip(line_data["lat"], line_data["lon"])]

# Calculate distances using MapQuest API and draw dotted lines with popups
for i in range(len(line_coords) - 1):
    distance = calculate_distance(line_coords[i], line_coords[i + 1])
    if distance > 0.0:
        line_coords_mapquest = [line_coords[i], line_coords[i + 1]]
        line_mapquest = folium.PolyLine(locations=line_coords_mapquest, color='red', weight=2, opacity=0.7, dash_array='5, 15')
        popup_text = f"Distance entre {line_data.iloc[i]['name']} et {line_data.iloc[i+1]['name']}: {distance:.2f} kilometers"
        line_mapquest.add_child(folium.Popup(popup_text)).add_to(my_map)

# Create a legend HTML
legend_html = """
    <div style="
        position: fixed;
        bottom: 50px;
        left: 50px;
        z-index: 1000;
        padding: 10px;
        border: 2px solid grey;
        background-color: white;
        font-size: 16px;
    ">
    <p><i class="fa fa-hospital" style="color:green; font-size: 30px;"></i>&nbsp;Pharmacie</p>
    <p><i class="fa fa-train" style="color:red; font-size: 30px;"></i>&nbsp;Gare</p>
    <p><i class="fa fa-graduation-cap" style="color:blue; font-size: 30px;"></i>&nbsp;YouCode</p>
    <p><i class="fa fa-mosque" style="color:orange; font-size: 30px;"></i>&nbsp;Masjid</p>
    <p><i class="fa fa-shopping-basket" style="color:purple; font-size: 30px;"></i>&nbsp;Souk</p>
    <p><i class="fa fa-hotel" style="color:brown; font-size: 30px;"></i>&nbsp;Hôtel</p>
    <p><i class="fa fa-tree" style="color:#66ff66; font-size: 30px;"></i>&nbsp;Jardin</p>
    </div>
"""

# Add the legend HTML to the map
my_map.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
my_map.save('map.html')
